Part 3: DNA binding classifier

again, help from Isaiah Hazelwood and ChatGPT 

loading prerequisites and data

In [ ]:
from nn import nn 
from nn import io 
from nn import preprocess
import sklearn
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
positive_seqs = io.read_text_file("./data/rap1-lieb-positives.txt")
negative_seqs = io.read_fasta_file("./data/yeast-upstream-1k-negative.fa")
#negative sequences are 1000 characters while positives are 17 characters long
#need to have same length of positive and negative examples
#need to randomly select 17-character long substring from each negative sequence
np.random.seed(42)
negative_seqs_trimmed = []
for negative_seq in negative_seqs:
    i = np.random.randint(len(negative_seqs) - 17 + 1) #random starting position
    negative_seqs_trimmed.append(negative_seq[i:i+17])

#generate a full sequence list 
seqs = positive_seqs + negative_seqs_trimmed
labels = ([True] * len(positive_seqs)) + ([False] * len(negative_seqs_trimmed))

#randomly sample sequences to account for class imbalance 
sampled_seqs, sampled_labels = preprocess.sample_seqs(seqs, labels)

#now I'm encoding the sequences
sample_seqs_encoded = preprocess.one_hot_encode_seqs(sampled_seqs)

#randomly split sequences into training and validation sets
indexes_permuted = np.random.permutation(len(sampled_labels))
train_indices = indexes_permuted[:len(sample_seqs_encoded) * 7 // 10]
val_indices = indexes_permuted[len(sample_seqs_encoded) * 7 // 10:]
seqs_train = np.array([sample_seqs_encoded[i] for i in train_indices]).reshape((-1, 68, 1))